In [161]:
from lxml import html
import io, json
from os import walk
import re

In [164]:
#_, _, filenames = next(walk("req/dravidaveda"), (None, None, []))
_, _, filenames = next(walk("req1"), (None, None, []))

In [165]:
#wb = open('msongsDetails.json', 'w+')
wb = io.open('req1.json', 'w', encoding='utf-8')
output = []
# exc = open('exception.txt', 'w')
# suc = open('success.txt', 'w')

for file in filenames:
    page = open("req1/" + file).read()
#    page = open("req1/index.php?option=com_content&view=article&id=1290").read()
#    page = open("req/dravidaveda/" + file).read()
    tree = html.fromstring(page)
    #try: 
    number = tree.xpath('//div[@class="art-article"]//h5//span//text()')
    if len(number) == 0:
        number = tree.xpath('//div[@class="art-article"]//h5//text()')
    numstr = ''.join(number)       
    numint = int(filter(str.isdigit, numstr))

    sloka = tree.xpath('//div[@class="art-article"]//p[@style="text-align: left; padding-left: 150px;"]//text()')
    if len(sloka) == 0:
        sloka = tree.xpath('//div[@class="art-article"]//p[@style="padding-left: 150px;"]/text()')
    if len(sloka) == 0:
        sloka = tree.xpath('//div[@class="art-article"]//p[@style="padding-left: 180px;"]/text()')

    sloka = filter(lambda(x): x != ' ' and x != u"\xa0" and x != ', ', sloka)
    sloka = map(lambda(x): " ".join(x.split()), sloka)
    table_words = []
    table_meanings = []
    table = tree.xpath('//div[@class="art-article"]//table//tbody//tr')
    for row in table:
        word = ''
        meaning = ''
        it = list(row.iter("td"))
        if len(it) == 3:
            if len(it[0]) > 0:
                word = list(it[0].itertext())[1]
            table_words.append(word)
            if len(it[2]) > 0:
                meaning = list(it[2].itertext())[1]
            table_meanings.append(meaning)
        elif len(it) == 1:
            if len(it[0]) > 0:
                word = list(it[0].itertext())[1]
            table_words.append(word)
            table_meanings.append(word)
        elif len(it) == 2:
            if len(it[0]) > 0:
                word = list(it[0].itertext())[1]
            if len(it[1]) > 0:
                meaning = list(it[1].itertext())[1]
            table_words.append(word)
            table_meanings.append(meaning) 
            print word
            print meaning

    table_words = map(lambda(x): " ".join(x.split()), table_words)
    table_meanings = map(lambda(x): " ".join(x.split()), table_meanings)
            
    total1 =  tree.xpath('//div[@class="art-article"]//p[@style="text-align: justify;"]//text()')

    total2 =  tree.xpath('//div[@class="art-article"]//p//span[@style="text-align: justify;"]//text()')
    total3 =  tree.xpath('//div[@class="art-article"]//p//span[@style="font-size: small;"]//text()')
     

    total = filter(lambda(x): x not in [", ", "; ", "- ", "? ", ". ", '', " "], total1 + total2 + total3)
    total = map(lambda(x): " ".join(x.split()), total)
    regex = re.compile(r'^[ -~]+$')
    tamil = []
    english = []
    for line in total:
#        print line
        newline = "".join(re.findall(r'[\w+]', line, re.UNICODE))
#         newline = "".join(line.split('\xa0“'), re.UNICODE)
#         print newline
        if regex.match(newline):
            english.append(line)
        elif newline != "":
            tamil.append(line)
#            print "tamil"
#    print english
    print("{:60}".format(file) + " *Id: {:5.5} ".format(str(numint)) + "sl: {:>2} ".format(len(sloka))
              + "Tw: {:>2} ".format(len(table_words)) + "Tm: {:>2} ".format(len(table_meanings))
              + "Te: {:>2} ".format(len(tamil)) + "Ee: {:>2} ".format(len(english)) + "\n")

#         print numint
#         print sloka
#         print table_words
#         print table_meanings
#         print tamil
#         print english

    obj = {
        "Id": numint,
        "TamSong": sloka,
        "TamTblWords": table_words,
        "TamTblMeanings": table_meanings,
        "TamExpln": tamil,
        "EngExpln": english
    }

    output.append(obj)

    #except Exception:
#         exc.write(file)
#         exc.write("\n")
     #   continue      
    
# print output
wb.write(json.dumps(output, ensure_ascii=False, sort_keys=True))
wb.close()
# exc.close()
# suc.close()

index.php?option=com_content&view=article&id=207&Itemid=61   *Id: 42    sl:  4 Tw: 16 Tm: 16 Te:  1 Ee:  1 

